# Load Senzing Truth-Sets

These instructions load the [Senzing Truth-Sets]

[Senzing Truth-Sets]: https://github.com/Senzing/truth-sets

## Identify data sources

To add Truth Set data sources to the Senzing configuration, copy/paste the following block of code into the interactive Python session and press the Enter key. Example:

Import packages.

- `TRUTHSET_DATASOURCES` contains the datasources used by the Truth Sets
- `TRUTHSET_CUSTOMER_RECORDS`, `TRUTHSET_REFERENCE_RECORDS`, and `TRUTHSET_WATCHLIST_RECORDS` are lists of records to be inserted.

In [ ]:
import grpc
from senzing_grpc import SzAbstractFactory, SzEngineFlags, SzError
from senzing_truthset import (
    TRUTHSET_CUSTOMER_RECORDS,
    TRUTHSET_DATASOURCES,
    TRUTHSET_REFERENCE_RECORDS,
    TRUTHSET_WATCHLIST_RECORDS,
)

Create Senzing object.

In [ ]:
sz_abstract_factory = SzAbstractFactory(
    grpc_channel=grpc.insecure_channel("localhost:8261")
)
sz_config = sz_abstract_factory.create_sz_config()
sz_configmanager = sz_abstract_factory.create_sz_configmanager()
sz_diagnostic = sz_abstract_factory.create_sz_diagnostic()
sz_engine = sz_abstract_factory.create_sz_engine()

Get existing Senzing configuration.

In [ ]:
old_config_id = sz_configmanager.get_default_config_id()
OLD_JSON_CONFIG = sz_configmanager.get_config(old_config_id)
config_handle = sz_config.import_config(OLD_JSON_CONFIG)  

Add DataSources to existing Senzing configuration.

In [ ]:
try:
    for datasource in TRUTHSET_DATASOURCES.keys():
        sz_config.add_data_source(config_handle, datasource)
except SzError as err:
    print(err)

Persist new Senzing configuration.

In [ ]:
NEW_JSON_CONFIG = sz_config.export_config(config_handle)
new_config_id = sz_configmanager.add_config(
    NEW_JSON_CONFIG, "Add TruthSet datasources"
)
sz_configmanager.replace_default_config_id(old_config_id, new_config_id)    

Update other Senzing objects

In [ ]:
sz_engine.reinitialize(new_config_id)
sz_diagnostic.reinitialize(new_config_id)

## Add records

Identify records

In [ ]:
record_sets = [
    TRUTHSET_CUSTOMER_RECORDS,
    TRUTHSET_REFERENCE_RECORDS,
    TRUTHSET_WATCHLIST_RECORDS,
]

Prime engine

In [ ]:
sz_engine.prime_engine()

Call Senzing to add records

In [ ]:
for record_set in record_sets:
    for record in record_set.values():
        info = sz_engine.add_record(
            record.get("DataSource"),
            record.get("Id"),
            record.get("Json"),
            SzEngineFlags.SZ_WITH_INFO,
        )
        print(info)